In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from random import random, randrange, choice, shuffle
from datetime import datetime, timedelta, date
from itertools import combinations
from pprint import pprint
import calendar
import plotly.express as px
import time

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

import warnings
warnings.filterwarnings('ignore')

from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all" # Cell의 모든 반환값 출력

pd.set_option('display.max_rows', None)

import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

# 함수 정의

## 블록데이터 전처리 함수

In [2]:
import functools
def unpack_df_columns(func):
    @functools.wraps(func)
    def _unpack_df_columns(*args, **kwargs):
        series = args[0]
        return func(*series.values)
    return _unpack_df_columns

@unpack_df_columns
def 최소착수요구일구하기(납기, 공기):
    result = pd.to_datetime(납기) - timedelta(days=int(공기))
    return result.date()

@unpack_df_columns
def 블록우선순위구하기(날짜순서, 공기순서, 크기순서):
    global 착수일가중치, 공기가중치, 크기가중치
    result = np.round((날짜순서*float(착수일가중치) + 공기순서*float(공기가중치) + 크기순서*float(크기가중치))/3,1)
    return result

@unpack_df_columns
def 블록사이즈튜플만들기(가로, 세로):
    길이2 = max(가로, 세로)  
    길이1 = min(가로, 세로)
    result = (길이2, 길이1)
    return result

def 블록데이터전처리(블록원데이터):
    df1 = 블록원데이터.copy()
    df1["blk_id"] = df1["블록명"].apply(lambda x: x[1:])
    df1["사이즈"] = df1[["가로", "세로"]].apply(블록사이즈튜플만들기, axis=1)
    df1["최대길이"] = df1["사이즈"].apply(lambda x: max(x))
    df1["납기"] = pd.to_datetime(df1["납기"])
    df1["면적"] = df1.eval("가로*세로")
    df1["최소착수요구일"] = df1[["납기", "표준공기"]].apply(최소착수요구일구하기, axis=1)
    df1["날짜순서"] = df1["최소착수요구일"].rank()
    df1["공기순서"] = df1["표준공기"].rank(ascending=False)
    df1["크기순서"] = df1["면적"].rank(ascending=False)
    df1["우선순위"] = df1[["날짜순서", "공기순서", "크기순서"]].apply(블록우선순위구하기, axis=1)
    
    df1 = df1.drop(['가로', '세로'], axis=1)
    df1 = df1.sort_values(by=["우선순위"])
    return df1

## 정반데이터 전처리 함수

In [3]:
@unpack_df_columns
def 정반우선순위구하기(중순, 크순):
    global 중량가중치, 크기가중치
    result = np.round((중순*float(중량가중치) + 크순*float(크기가중치))/3,1)
    return result

@unpack_df_columns
def 정반사이즈튜플만들기(가로, 세로):
    길이1 = max(가로, 세로)  
    길이2 = min(가로, 세로)
    result = (길이1, 길이2)
    return result

def 정반데이터전처리(정반원데이터):
    df = 정반원데이터.copy()
    
    df["사이즈"] = df[["가로", "세로"]].apply(정반사이즈튜플만들기, axis=1)
    df["면적"] = df.eval("가로*세로")
    df["최대길이"] = df["사이즈"].apply(lambda x: max(x))
    df["중량순서"] = df["가능중량"].rank(ascending=False)
    df["크기순서"] = df["면적"].rank(ascending=False)
    df["우선순위"] = df[["중량순서", "크기순서"]].apply(정반우선순위구하기, axis=1)
    
    df = df.drop(['가로', '세로'], axis=1)
    df = df.sort_values(by=["우선순위"])
    return df

## 물리조건 체크 함수 (중량, 최장길이)

In [4]:
def 중량조건체크(target_block):
    가능정반리스트 = []
    블록중량 = 블록데이터[블록데이터["블록명"]==target_block]["중량"].values[0]
    for 정반 in 정반데이터["정반명"]:
        정반가능중량 = 정반데이터[정반데이터["정반명"]==정반]["가능중량"].values[0]
        if 정반가능중량 > 블록중량:
            가능정반리스트.append(정반)       
    return 가능정반리스트  

def 최장길이체크(target_block:str, 중량가능정반들:list):
    가능정반리스트 = []
    블록최장길이 = 블록데이터[블록데이터["블록명"]==target_block]["최대길이"].values[0]
    for 정반 in 중량가능정반들:
        정반최장길이 = 정반데이터[정반데이터["정반명"]==정반]["가능중량"].values[0]
        if 정반최장길이 > 블록최장길이:
            가능정반리스트.append(정반)
    return 가능정반리스트

## 레이아웃 함수

In [5]:
def init_jungban(사이즈):  #사이즈 = (10, 10)
    surface_width, surface_height = 사이즈[0], 사이즈[1]  # Adjusted to match the provided image for demonstration
    surface = np.zeros((surface_height, surface_width), dtype=int)
    return surface, surface_width, surface_height

def can_place_with_thresh(surface, surface_width, surface_height, block_height, block_width, start_row, start_col, thresh):
    new_width = surface_width - thresh
    new_height = surface_height - thresh
    
    if start_row + block_height > surface_height or start_col + block_width > surface_width:
        return False

    block_area = surface[start_row:start_row+block_height, start_col:start_col+block_width]
    if np.any(block_area != 0):
        return False
    
    if start_row > 0 and np.any(surface[start_row-thresh: start_row, start_col:start_col+block_width] != 0):
        return False
    if start_col > 0 and np.any(surface[start_row: start_row+block_height, start_col-thresh: start_col] != 0):
        return False
    
    return True

# Function to place a block on the surface, if possible
def place_block(surface, block_height, block_width, start_row, start_col, block_id):
    block_height, block_width = block_height, block_width
    surface[start_row:start_row + block_height, start_col:start_col + block_width] = block_id
    return surface


# def 레이아웃배치(surface, surface_width, surface_height, block_width, block_height, block_id, thresh):
def 레이아웃배치(surface, surface_width, surface_height, block_height, block_width, block_id, thresh):

    best_fit_score = float('inf')
    best_position = None
    
    for start_row in range(surface_height - block_height + 1):
        for start_col in range(surface_width - block_width + 1):
            if can_place_with_thresh(surface, surface_width, surface_height, block_height, block_width, start_row, start_col, thresh):
                # Calculate a score; here we use the top-left corner (y, x) as the score
                # A lower score means the block is closer to the top-left
                score = start_row + start_col
                if score < best_fit_score:
                    best_fit_score = score
                    best_position = (start_row, start_col)
    
    if best_position:
        surface = place_block(surface, block_height, block_width, *best_position, block_id)
        return True, surface
    else:
        return False, surface

def get_overlapped_surface(*arrays):
    result = arrays[0]
    for arr in arrays[1:]:
        result = np.add(result, arr)
    return result

# 특정 값만 남기고 다 지운후에 그 결과를 다시 각 해당일 surface에 더해준다...천재네..ㅋ
def extract_newblock_layout_only(overlapped_surface, block_id):
    mask = overlapped_surface != block_id
    overlapped_surface[mask] = 0
    newblock_layout = overlapped_surface
    return newblock_layout

In [6]:
# def draw_layout(레이아웃달력, 정반, idx):
#     target_date = 레이아웃달력.index[idx]
#     target_surface = 레이아웃달력.at[target_date, 정반]
#     fig, ax = plt.subplots(figsize=(8, 6))
#     plt.imshow(target_surface, cmap="Blues", interpolation='nearest')
    
#     for i in range(len(target_surface)):
#         for j in range(len(target_surface[0])):
#             plt.text(j, i, f'{target_surface[i, j]}', ha='center', va='center', color='white', fontsize=7)
#     # plt.colorbar()  # Add a color bar to show the mapping of values to colors
#     plt.show()

In [7]:
def draw_layout_subplots(레이아웃달력, 정반리스트, idx):
    정반1 = 정반리스트[0]
    정반2 = 정반리스트[1]
    
    target_date = 레이아웃달력.index[idx]
    target_surface1 = 레이아웃달력.at[target_date, 정반1]
    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))  # Create a figure with 2 subplots
    
    # Plot the first subplot
    ax = axs[0]
    ax.imshow(target_surface1, cmap="Blues", interpolation='nearest')
    for i in range(len(target_surface1)):
        for j in range(len(target_surface1[0])):
            ax.text(j, i, f'{target_surface1[i, j]}', ha='center', va='center', color='white', fontsize=6)
    ax.set_title(f'{정반1} - First Subplot')  # Set subplot title
    
    # Plot the second subplot
    target_surface2 = 레이아웃달력.at[target_date, 정반2]
    ax = axs[1]
    ax.imshow(target_surface2, cmap="Blues", interpolation='nearest')
    for i in range(len(target_surface2)):
        for j in range(len(target_surface2[0])):
            ax.text(j, i, f'{target_surface2[i, j]}', ha='center', va='center', color='white', fontsize=6)
    ax.set_title(f'{정반2} - Second Subplot')  # Set subplot title
    
    plt.tight_layout()  # Adjust subplot layout
    plt.show()

In [17]:
def 기배치블록배치레이아웃(정반사이즈, 좌상단꼭지점좌표, 블록사이즈, blk_id):
    정반어레이 = np.zeros((정반사이즈[0], 정반사이즈[1]))
    x, y = 좌상단꼭지점좌표
    x = max(x-1, 0)
    y = max(y-1, 0)
    for i in range(블록사이즈[1]):
        for j in range(블록사이즈[0]):
            정반어레이[i+x, j+y] = int(blk_id)   
    return 정반어레이   

## 레이아웃 달력

In [8]:
def get_end_date_of_month(year, month):
    # Get the number of days in the given month
    num_days = calendar.monthrange(year, month)[1]
    # Return the last day of the month as a datetime object
    return date(year, month, num_days)

def create_레이아웃달력(시작년:int, 시작월:int, 시작일: int, 종료년:int, 종료월:int, 정반데이터:pd.DataFrame) -> pd.DataFrame:
    start_date = datetime(시작년, 시작월, 시작일)
    end_date = get_end_date_of_month(종료년, 종료월)
    정반집합 = 정반데이터["정반명"].tolist()
    날짜집합  = pd.date_range(start=start_date, end=end_date, freq='D')
    
    달력 = pd.DataFrame()
    달력.index = 날짜집합
    
    for 정반 in 정반집합:
        정반사이즈 = 정반데이터[정반데이터["정반명"]==정반]["사이즈"].values[0]
        # print(f"{정반}정반사이즈: {정반사이즈}")
        surface, surface_width, surface_height = init_jungban(정반사이즈)
        달력[정반] = [surface for _ in range(len(날짜집합))]
        
    return 달력

In [32]:
def update_레이아웃달력(레이아웃달력, 정반명, 블록명, 정반데이터, 블록데이터, 조기착수금지일, thresh, 후퇴배수):
    # 변수정리
    # 조기착수금지일 = 7    # 자재준비기간 고려한 최대조기착수 한계일수
    # thresh = 1   # 블록사이간격 (작업성고려 최소 1미터)
    최소착수요구일 = 블록데이터[블록데이터["블록명"]==블록명]["최소착수요구일"].values[0]
    표준공기 = int(블록데이터[블록데이터["블록명"]==블록명]["표준공기"].values[0])
    blk_id = 블록데이터[블록데이터["블록명"]==블록명]["blk_id"].values[0]
    블록사이즈 = 블록데이터[블록데이터["블록명"]==블록명]["사이즈"].values[0]
    정반사이즈 = 정반데이터[정반데이터["정반명"]==정반명]["사이즈"].values[0]
    surface_width, surface_height = 정반사이즈[0], 정반사이즈[1]
    block_width, block_height = 블록사이즈[0], 블록사이즈[1]
    
    날짜리스트 = [d.strftime("%Y-%m-%d") for d in 레이아웃달력.index]
    start_date = 최소착수요구일 # datetime.strptime(최소착수요구일, "%Y-%m-%d").date()
    min_start_date = start_date - timedelta(days=조기착수금지일)
    
    end_date = min_start_date + timedelta(days=표준공기)
    min_start_date= min_start_date.strftime('%Y-%m-%d')
    end_date = end_date.strftime('%Y-%m-%d')
    min_start_idx = 날짜리스트.index(min_start_date)
    end_idx = 날짜리스트.index(end_date)
    
    # 레이아웃 순환 검토 
    # 후퇴배수 = int(표준공기 * 1)   # 최선조기착수일 인덱스에서 표준공기의 최대 00배수까지 후퇴 검토
    # 후퇴배수 = 14 # 그냥 숫자로 한정하기
    
    for k in range(후퇴배수):  
        # min_start_idx += 1
        # end_idx += 1
        print(f">>>>> 검토차수{k} / {레이아웃달력.index[min_start_idx]}: {min_start_idx} ~ {end_idx}")
    
        target_surfaces = []
        temp_dates = []
        for idx in range(min_start_idx, end_idx+1): # 표준공기에 해당하는 시기인덱스~종기인덱스 순환 검토
            target_date = 레이아웃달력.index[idx]    # 대상인덱스의 날짜 불러오기
            temp_dates.append(target_date)
            target_surface = 레이아웃달력.at[target_date, 정반명]  # 조회날짜의 레이아웃(서페이스) 불러오기
            target_surfaces.append(target_surface)   # 조회날짜의 레이아웃들을 리스트로 담기
                
        overlapped_surface = get_overlapped_surface(*target_surfaces)  # 리스트에 담은 레이아웃을 합하여 오버랩 레이아웃 생성
        
        bool_res, surface = 레이아웃배치(overlapped_surface, surface_width, surface_height, block_width, block_height, blk_id, thresh)  # 오버랩 레이아웃에 신규 블록 레이아웃 배치검토
        
        if bool_res:  # 신규블록이 배치되어 True라면...
            newblock_layout = extract_newblock_layout_only(surface, int(blk_id))  # 신규배치블록의 레이아웃만 추출 (기배치 다른 블록 레이아웃 제거)
            
            for idx in range(min_start_idx, end_idx+1):   # 조회기간 다시 탐색 (레이아웃 업데이트를 위해)
                target_date = 레이아웃달력.index[idx]  
                target_surface = 레이아웃달력.at[target_date, 정반명]    # 조회일의 레이아웃 불러오기
                target_surface = target_surface + newblock_layout        # 조회일의 레이아웃에 새블록 레이아웃 더하여 내용 합치기
                레이아웃달력.at[target_date, 정반명] = target_surface   # 해당일의 레이아웃을 새레이아웃으로 변경하기 (업데이트 완료)
            
            최선배치일 = temp_dates[0].strftime('%Y-%m-%d')
            최선정반 = 정반명
            break

        else:
            # 가로세로 바꿔서
            bool_res, surface = 레이아웃배치(overlapped_surface, surface_width, surface_height, block_height, block_width, blk_id, thresh)  

            if bool_res:
                newblock_layout = extract_newblock_layout_only(surface, int(blk_id))  # 신규배치블록의 레이아웃만 추출 (기배치 다른 블록 레이아웃 제거)
            
                for idx in range(min_start_idx, end_idx+1):   # 조회기간 다시 탐색 (레이아웃 업데이트를 위해)
                    target_date = 레이아웃달력.index[idx]  
                    target_surface = 레이아웃달력.at[target_date, 정반명]    # 조회일의 레이아웃 불러오기
                    target_surface = target_surface + newblock_layout        # 조회일의 레이아웃에 새블록 레이아웃 더하여 내용 합치기
                    레이아웃달력.at[target_date, 정반명] = target_surface   # 해당일의 레이아웃을 새레이아웃으로 변경하기 (업데이트 완료)
            
                최선배치일 = temp_dates[0].strftime('%Y-%m-%d')
                최선정반 = 정반명
                break

            else:  # 신규블록 배치가 False라면.. 계속 순환
                최선배치일 = None
                최선정반 = None

        min_start_idx += 1
        end_idx += 1
        
        
    return 레이아웃달력, 최선정반, 최선배치일

In [33]:
def find_earlier(data):
    try:
        data = {key: value for key, value in data.items() if value is not None}
        earliest_date = min(data.values())
        earliest_items = [(key, value) for key, value in data.items() if value == earliest_date]
        earliest_item = choice(earliest_items)
        return earliest_item
    except:
        return None, None

In [34]:
# 레이아웃달력 = create_레이아웃달력(2024, 2, 1, 2024, 4, 정반데이터)
# 대상정반 = "A"
# 정반사이즈 = (42, 28)
# 최소착수요구일 = "2024-02-10"
# 조기착수금지일 = 7
# 표준공기 = 3
# 레이아웃달력 = 레이아웃달력
# 블록사이즈 = (20, 20)
# blk_id = 1
# thresh = 1 # 블록최소간격

# surface_width, surface_height = 정반사이즈[0], 정반사이즈[1]
# block_width, block_height = 블록사이즈[0], 블록사이즈[1]

# 날짜리스트 = [d.strftime("%Y-%m-%d") for d in 레이아웃달력.index]
# start_date = datetime.strptime(최소착수요구일, "%Y-%m-%d").date()
# min_start_date = start_date - timedelta(days=조기착수금지일)

# end_date = min_start_date + timedelta(days=표준공기)
# min_start_date= min_start_date.strftime('%Y-%m-%d')
# end_date = end_date.strftime('%Y-%m-%d')
# min_start_idx = 날짜리스트.index(min_start_date)
# end_idx = 날짜리스트.index(end_date)
# min_start_date, min_start_idx, end_idx

In [35]:
# for k in range(표준공기*3):
#     min_start_idx += k
#     end_idx += k
#     print(f">>> 검토차수{k}: {min_start_idx} ~ {end_idx}")

#     target_surfaces = []
#     temp_dates = []
#     for idx in range(min_start_idx, end_idx+1):
#         target_date = 레이아웃달력.index[idx]
#         temp_dates.append(target_date)
#         target_surface = 레이아웃달력.at[target_date, 대상정반]
#         target_surfaces.append(target_surface)

#     overlapped_surface = get_overlapped_surface(*target_surfaces)
#     bool_res, surface = 레이아웃배치(overlapped_surface, surface_width, surface_height, block_width, block_height, blk_id, thresh)
    
#     if bool_res:
#         print(">>> 조회기간 레이아웃 배치가능")
#         newblock_layout = extract_newblock_layout_only(surface, blk_id)

#         for idx in range(min_start_idx, end_idx+1):
#             target_date = 레이아웃달력.index[idx]
#             target_surface = 레이아웃달력.at[target_date, 대상정반]
#             target_surface = target_surface + newblock_layout
#             레이아웃달력.at[target_date, 대상정반] = target_surface
#             print(idx, len(target_surface))      
#         break
#     else:
#         print(">>> 다음차수검토")
# 레이아웃달력.head(8)

# 시뮬레이션

## 최선가능 루프

In [36]:
# data_num = "_실행계획3"
# 블록원데이터 = pd.read_excel(f"./data/data1{data_num}.xlsx", sheet_name="블록데이터")
# 기배치블록 = pd.read_excel(f"./data/data1{data_num}.xlsx", sheet_name="기배치블록")
# 정반원데이터 = pd.read_excel(f"./data/data1{data_num}.xlsx", sheet_name="정반데이터")
# 블록원데이터.shape

data_num = "_실행계획"
블록원데이터 = pd.read_excel(f"D:/공정최적화/data/data1{data_num}.xlsx", sheet_name="블록데이터")
기배치블록 = pd.read_excel(f"D:/공정최적화/data/data1{data_num}.xlsx", sheet_name="기배치블록")
정반원데이터 = pd.read_excel(f"D:/공정최적화/data/data1{data_num}.xlsx", sheet_name="정반데이터")

기배치블록.rename(columns={'공기':'표준공기'}, inplace=True)
기배치블록["blk_id"] = 기배치블록["블록명"].apply(lambda x: x[1:])

기배치블록1 =기배치블록[["블록명", "중량", "가로", "세로", "표준공기", "종료일"]]
기배치블록1.rename(columns={'종료일':'납기'}, inplace=True)
블록마스터 = pd.concat([기배치블록1, 블록원데이터])

검토개수 = 115
블록원데이터 = 블록원데이터.iloc[:검토개수,:]
블록원데이터.shape

착수일가중치, 공기가중치, 크기가중치 = 0.9, 0.5, 0.5
블록데이터 = 블록데이터전처리(블록원데이터)

중량가중치, 크기가중치 = 0.5, 0.5
정반데이터 = 정반데이터전처리(정반원데이터)
블록마스터.shape, 기배치블록.shape, 블록데이터.shape, 정반데이터.shape

# -----------------------------------------------------------------------------------------
시작년, 시작월, 시작일, 종료년, 종료월 = 2023, 10, 1, 2024, 5
레이아웃달력 = create_레이아웃달력(시작년, 시작월, 시작일, 종료년, 종료월, 정반데이터)

#-------------------------------------------------------------------------------------------------------------
미배치결과 = []
델타결과 = []
for _ in tqdm(range(1)):
    # 적당히 배치되게 가정함
    꼭지점 = [(0, 0), (7, 0), (0, 0), (9, 0), (17, 0), (13, 0), (0, 0), (26, 0), (22, 0), (0, 12), (0, 16), (16, 16),]

    기배치블록["꼭지점"] = 꼭지점
    기배치블록["사이즈"] = 기배치블록[["가로","세로"]].apply(블록사이즈튜플만들기, axis=1)

    시작년, 시작월, 시작일, 종료년, 종료월 = 2023, 10, 1, 2024, 5
    레이아웃달력 = create_레이아웃달력(시작년, 시작월, 시작일, 종료년, 종료월, 정반데이터)
    날짜리스트 = [d.strftime("%Y-%m-%d") for d in 레이아웃달력.index]

    for 블록명 in 기배치블록["블록명"]:
        # 변수 정의
        정반명 = 기배치블록[기배치블록["블록명"]==블록명]["배치정반"].values[0]
        정반사이즈 = 정반데이터[정반데이터["정반명"]==정반명]["사이즈"].values[0]
        좌상단꼭지점좌표 = 기배치블록[기배치블록["블록명"]==블록명]["꼭지점"].values[0]
        블록사이즈 = 기배치블록[기배치블록["블록명"]==블록명]["사이즈"].values[0]
        blk_id = 기배치블록[기배치블록["블록명"]==블록명]["blk_id"].values[0]

        # 기배치 블록 레이아웃 검토
        newblock_layout = 기배치블록배치레이아웃(정반사이즈, 좌상단꼭지점좌표, 블록사이즈, blk_id)

        # 레이아웃 달력 업데이트 변수 정의
        착수일 = 기배치블록[기배치블록["블록명"]==블록명]["착수일"].values[0]
        종료일 = 기배치블록[기배치블록["블록명"]==블록명]["종료일"].values[0]
        착수일 = np.datetime_as_string(착수일, unit='D')
        종료일 = np.datetime_as_string(종료일, unit='D')
        착수일_idx = 날짜리스트.index(착수일)
        종료일_idx = 날짜리스트.index(종료일)

        # 레이아웃 달력 업데이트
        for idx in range(착수일_idx, 종료일_idx+1):
            target_date = 레이아웃달력.index[idx]  
            target_surface = 레이아웃달력.at[target_date, 정반명]
            newblock_layout1 = np.transpose(newblock_layout)
            target_surface = target_surface + newblock_layout1   
            레이아웃달력.at[target_date, 정반명] = target_surface

    블록리스트 = 블록데이터["블록명"].tolist()
    정반리스트 = 정반데이터["정반명"].tolist()

    배치정반모음 = []
    배치일모음 = []
#--------------------------------------------------------------------------------------------------------------
    for _ in tqdm(range(len(블록리스트))):
        target_block = 블록리스트[0]
        print(f"***** 검토대상블록: {target_block} ---------------------------------------------")

        # 물리조건 체크
        가능정반들 = 중량조건체크(target_block)
        가능정반들 = 최장길이체크(target_block, 가능정반들)
        
        print(f"물리조건가능정반: {가능정반들}")

        조기착수금지, 블록간격, 후퇴계수 = 7, 1, 10 ##################
       
        최종배치일모음 = dict()
        prev_레이아웃달력 = 레이아웃달력.copy()
            
        for 최선정반 in 가능정반들:
            레이아웃달력, 최선정반, 최종배치일 = update_레이아웃달력(prev_레이아웃달력, 최선정반, target_block, 정반데이터, 블록데이터, 조기착수금지, 블록간격, 후퇴계수) 
            최종배치일모음[최선정반] = 최종배치일
        
        print(f"최종배치일후보: {최종배치일모음}")
        최선정반, 최종배치일 = find_earlier(최종배치일모음)

        배치정반모음.append(최선정반)
        배치일모음.append(최종배치일)
        블록리스트.remove(target_block)
        
        print(f"최종배치결과: {최선정반} - {최종배치일}")

    배치후데이터 = 블록데이터.copy()
    배치후데이터["배치정반"] = 배치정반모음
    배치후데이터["착수일자"] = 배치일모음
    # 최소착수요구일 대비 착수일자 차이(델타) 계산
    배치후데이터['최소착수요구일'] = pd.to_datetime(배치후데이터['최소착수요구일'])
    배치후데이터['착수일자'] = pd.to_datetime(배치후데이터['착수일자'])
    배치후데이터["델타"] = 배치후데이터['착수일자'] - 배치후데이터['최소착수요구일']
    배치후데이터["델타"] = 배치후데이터["델타"].dt.days
    배치후데이터.to_csv("./results/배치결과_8_3.csv", encoding='utf-8-sig')
    
    result = dict()
    for key, value in zip(배치후데이터["델타"].value_counts().sort_index().index, 배치후데이터["델타"].value_counts().sort_index().values):
        result[key] = value
  
    미배치결과.append(배치후데이터['착수일자'].isnull().sum())
    델타결과.append(result)

미배치결과

(115, 6)

((135, 6), (12, 9), (115, 13), (2, 8))

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/115 [00:00<?, ?it/s]

***** 검토대상블록: S33 ---------------------------------------------
물리조건가능정반: ['B', 'A']
>>>>> 검토차수0 / 2023-11-07 00:00:00: 37 ~ 50
>>>>> 검토차수1 / 2023-11-08 00:00:00: 38 ~ 51
>>>>> 검토차수0 / 2023-11-07 00:00:00: 37 ~ 50
최종배치일후보: {'B': '2023-11-08', 'A': '2023-11-07'}
최종배치결과: A - 2023-11-07
***** 검토대상블록: S24 ---------------------------------------------
물리조건가능정반: ['B', 'A']
>>>>> 검토차수0 / 2023-11-02 00:00:00: 32 ~ 44
>>>>> 검토차수1 / 2023-11-03 00:00:00: 33 ~ 45
>>>>> 검토차수2 / 2023-11-04 00:00:00: 34 ~ 46
>>>>> 검토차수3 / 2023-11-05 00:00:00: 35 ~ 47
>>>>> 검토차수4 / 2023-11-06 00:00:00: 36 ~ 48
>>>>> 검토차수5 / 2023-11-07 00:00:00: 37 ~ 49
>>>>> 검토차수6 / 2023-11-08 00:00:00: 38 ~ 50
>>>>> 검토차수7 / 2023-11-09 00:00:00: 39 ~ 51
>>>>> 검토차수8 / 2023-11-10 00:00:00: 40 ~ 52
>>>>> 검토차수9 / 2023-11-11 00:00:00: 41 ~ 53
>>>>> 검토차수0 / 2023-11-02 00:00:00: 32 ~ 44
>>>>> 검토차수1 / 2023-11-03 00:00:00: 33 ~ 45
>>>>> 검토차수2 / 2023-11-04 00:00:00: 34 ~ 46
>>>>> 검토차수3 / 2023-11-05 00:00:00: 35 ~ 47
>>>>> 검토차수4 / 2023-11-06 00:

[57]

## 랜덤 최선

In [37]:
# data_num = "_실행계획3"
# 블록원데이터 = pd.read_excel(f"./data/data1{data_num}.xlsx", sheet_name="블록데이터")
# 기배치블록 = pd.read_excel(f"./data/data1{data_num}.xlsx", sheet_name="기배치블록")
# 정반원데이터 = pd.read_excel(f"./data/data1{data_num}.xlsx", sheet_name="정반데이터")
# 블록원데이터.shape

data_num = "_실행계획"
블록원데이터 = pd.read_excel(f"D:/공정최적화/data/data1{data_num}.xlsx", sheet_name="블록데이터")
기배치블록 = pd.read_excel(f"D:/공정최적화/data/data1{data_num}.xlsx", sheet_name="기배치블록")
정반원데이터 = pd.read_excel(f"D:/공정최적화/data/data1{data_num}.xlsx", sheet_name="정반데이터")

기배치블록.rename(columns={'공기':'표준공기'}, inplace=True)
기배치블록["blk_id"] = 기배치블록["블록명"].apply(lambda x: x[1:])

기배치블록1 =기배치블록[["블록명", "중량", "가로", "세로", "표준공기", "종료일"]]
기배치블록1.rename(columns={'종료일':'납기'}, inplace=True)
블록마스터 = pd.concat([기배치블록1, 블록원데이터])

검토개수 = 115
블록원데이터 = 블록원데이터.iloc[:검토개수,:]
블록원데이터.shape

착수일가중치, 공기가중치, 크기가중치 = 0.9, 0.5, 0.5
블록데이터 = 블록데이터전처리(블록원데이터)

중량가중치, 크기가중치 = 0.5, 0.5
정반데이터 = 정반데이터전처리(정반원데이터)
블록마스터.shape, 기배치블록.shape, 블록데이터.shape, 정반데이터.shape

# -----------------------------------------------------------------------------------------
시작년, 시작월, 시작일, 종료년, 종료월 = 2023, 10, 1, 2024, 5
레이아웃달력 = create_레이아웃달력(시작년, 시작월, 시작일, 종료년, 종료월, 정반데이터)

#-------------------------------------------------------------------------------------------------------------
미배치결과 = []
델타결과 = []
for _ in tqdm(range(1)):
    # 적당히 배치되게 가정함
    꼭지점 = [(0, 0), (7, 0), (0, 0), (9, 0), (17, 0), (13, 0), (0, 0), (26, 0), (22, 0), (0, 12), (0, 16), (16, 16),]

    기배치블록["꼭지점"] = 꼭지점
    기배치블록["사이즈"] = 기배치블록[["가로","세로"]].apply(블록사이즈튜플만들기, axis=1)

    시작년, 시작월, 시작일, 종료년, 종료월 = 2023, 10, 1, 2024, 5
    레이아웃달력 = create_레이아웃달력(시작년, 시작월, 시작일, 종료년, 종료월, 정반데이터)
    날짜리스트 = [d.strftime("%Y-%m-%d") for d in 레이아웃달력.index]

    for 블록명 in 기배치블록["블록명"]:
        # 변수 정의
        정반명 = 기배치블록[기배치블록["블록명"]==블록명]["배치정반"].values[0]
        정반사이즈 = 정반데이터[정반데이터["정반명"]==정반명]["사이즈"].values[0]
        좌상단꼭지점좌표 = 기배치블록[기배치블록["블록명"]==블록명]["꼭지점"].values[0]
        블록사이즈 = 기배치블록[기배치블록["블록명"]==블록명]["사이즈"].values[0]
        blk_id = 기배치블록[기배치블록["블록명"]==블록명]["blk_id"].values[0]

        # 기배치 블록 레이아웃 검토
        newblock_layout = 기배치블록배치레이아웃(정반사이즈, 좌상단꼭지점좌표, 블록사이즈, blk_id)

        # 레이아웃 달력 업데이트 변수 정의
        착수일 = 기배치블록[기배치블록["블록명"]==블록명]["착수일"].values[0]
        종료일 = 기배치블록[기배치블록["블록명"]==블록명]["종료일"].values[0]
        착수일 = np.datetime_as_string(착수일, unit='D')
        종료일 = np.datetime_as_string(종료일, unit='D')
        착수일_idx = 날짜리스트.index(착수일)
        종료일_idx = 날짜리스트.index(종료일)

        # 레이아웃 달력 업데이트
        for idx in range(착수일_idx, 종료일_idx+1):
            target_date = 레이아웃달력.index[idx]  
            target_surface = 레이아웃달력.at[target_date, 정반명]
            newblock_layout1 = np.transpose(newblock_layout)
            target_surface = target_surface + newblock_layout1   
            레이아웃달력.at[target_date, 정반명] = target_surface

    블록리스트 = 블록데이터["블록명"].tolist()
    정반리스트 = 정반데이터["정반명"].tolist()

    배치정반모음 = []
    배치일모음 = []
#--------------------------------------------------------------------------------------------------------------
    for _ in tqdm(range(len(블록리스트))):
        target_block = 블록리스트[0]
        print(f"***** 검토대상블록: {target_block} ---------------------------------------------")

        # 물리조건 체크
        가능정반들 = 중량조건체크(target_block)
        가능정반들 = 최장길이체크(target_block, 가능정반들)
        
        print(f"물리조건가능정반: {가능정반들}")

        최선정반 = choice(가능정반들)

        조기착수금지, 블록간격, 후퇴계수 = 7, 1, 10 ##################
        레이아웃달력, 최선정반, 최종배치일 = update_레이아웃달력(레이아웃달력, 최선정반, target_block, 정반데이터, 블록데이터, 조기착수금지, 블록간격, 후퇴계수) 

        배치정반모음.append(최선정반)
        배치일모음.append(최종배치일)
        블록리스트.remove(target_block)
        
        print(f"최종배치결과: {최선정반} - {최종배치일}")

    배치후데이터 = 블록데이터.copy()
    배치후데이터["배치정반"] = 배치정반모음
    배치후데이터["착수일자"] = 배치일모음
    # 최소착수요구일 대비 착수일자 차이(델타) 계산
    배치후데이터['최소착수요구일'] = pd.to_datetime(배치후데이터['최소착수요구일'])
    배치후데이터['착수일자'] = pd.to_datetime(배치후데이터['착수일자'])
    배치후데이터["델타"] = 배치후데이터['착수일자'] - 배치후데이터['최소착수요구일']
    배치후데이터["델타"] = 배치후데이터["델타"].dt.days
    배치후데이터.to_csv("./results/배치결과_8_3.csv", encoding='utf-8-sig')
    
    result = dict()
    for key, value in zip(배치후데이터["델타"].value_counts().sort_index().index, 배치후데이터["델타"].value_counts().sort_index().values):
        result[key] = value
  
    미배치결과.append(배치후데이터['착수일자'].isnull().sum())
    델타결과.append(result)

미배치결과

(115, 6)

((135, 6), (12, 9), (115, 13), (2, 8))

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/115 [00:00<?, ?it/s]

***** 검토대상블록: S33 ---------------------------------------------
물리조건가능정반: ['B', 'A']
>>>>> 검토차수0 / 2023-11-07 00:00:00: 37 ~ 50
>>>>> 검토차수1 / 2023-11-08 00:00:00: 38 ~ 51
최종배치결과: B - 2023-11-08
***** 검토대상블록: S24 ---------------------------------------------
물리조건가능정반: ['B', 'A']
>>>>> 검토차수0 / 2023-11-02 00:00:00: 32 ~ 44
>>>>> 검토차수1 / 2023-11-03 00:00:00: 33 ~ 45
>>>>> 검토차수2 / 2023-11-04 00:00:00: 34 ~ 46
>>>>> 검토차수3 / 2023-11-05 00:00:00: 35 ~ 47
>>>>> 검토차수4 / 2023-11-06 00:00:00: 36 ~ 48
>>>>> 검토차수5 / 2023-11-07 00:00:00: 37 ~ 49
>>>>> 검토차수6 / 2023-11-08 00:00:00: 38 ~ 50
>>>>> 검토차수7 / 2023-11-09 00:00:00: 39 ~ 51
>>>>> 검토차수8 / 2023-11-10 00:00:00: 40 ~ 52
>>>>> 검토차수9 / 2023-11-11 00:00:00: 41 ~ 53
최종배치결과: B - 2023-11-11
***** 검토대상블록: S15 ---------------------------------------------
물리조건가능정반: ['B', 'A']
>>>>> 검토차수0 / 2023-10-27 00:00:00: 26 ~ 36
>>>>> 검토차수1 / 2023-10-28 00:00:00: 27 ~ 37
>>>>> 검토차수2 / 2023-10-29 00:00:00: 28 ~ 38
>>>>> 검토차수3 / 2023-10-30 00:00:00: 29 ~ 39
>>>>> 검토차수4

[28]